### Transformação dos dados

Nesse notebook vamos transformar os dados para identificar o momento (ano) em que uma data escola entrou no PEI.

In [1]:
import pandas as pd

from utils.load_csv import load_csv


In [2]:
df = load_csv('dados_merged.csv')

In [3]:
df.head()

,ano_censo,sigla_uf,nome_municipio,codigo_municipio,codigo_escola,nome_escola,qtd_turmas_ed_basica,qtd_matr_ed_basica,qtd_matr_medio,qtd_matr_fundamental,...,qtd_matriculas_infantil_integral,qtd_matriculas_infantil_creche_integral,qtd_matriculas_infantil_pre_escolar_integral,qtd_matriculas_fundamental_integral,qtd_matriculas_fundamental_anos_iniciais_integral,qtd_matriculas_fundamental_anos_finais_integral,qtd_matriculas_medio_integral,coordenadas,is_PEI,_merge
0,2011,SP,adamantina,3500105,35030806,helen keller,25,830,447,303,...,0,0,0,303,0,303,0,"['-21.693', '-51.0689']",True,both
1,2011,SP,adamantina,3500105,35031045,durvalino grion prof,22,771,275,496,...,0,0,0,0,0,0,0,"['-21.6786', '-51.0769']",True,both
2,2011,SP,adamantina,3500105,35031112,fleurides cavallini menechino profa,32,1102,345,757,...,0,0,0,0,0,0,1,"['-21.6928', '-51.0703']",True,both
3,2011,SP,adolfo,3500204,35027352,odila bovolenta de mendonca profa,6,156,156,0,...,0,0,0,0,0,0,0,NaN,False,left_only
4,2011,SP,aguai,3500303,35020527,geraldo lourenco padre,33,1106,508,598,...,0,0,0,0,0,0,0,NaN,False,left_only


In [4]:
df.columns

Index(['ano_censo', 'sigla_uf', 'nome_municipio', 'codigo_municipio',
       'codigo_escola', 'nome_escola', 'qtd_turmas_ed_basica',
       'qtd_matr_ed_basica', 'qtd_matr_medio', 'qtd_matr_fundamental',
       'qtd_matr_ed_basica_nao_declarada', 'qtd_matr_ed_basica_branca',
       'qtd_matr_ed_basica_preta', 'qtd_matr_ed_basica_parda',
       'qtd_matr_ed_basica_amarela', 'qtd_matr_ed_basica_indigena',
       'qtd_matriculas_infantil_integral',
       'qtd_matriculas_infantil_creche_integral',
       'qtd_matriculas_infantil_pre_escolar_integral',
       'qtd_matriculas_fundamental_integral',
       'qtd_matriculas_fundamental_anos_iniciais_integral',
       'qtd_matriculas_fundamental_anos_finais_integral',
       'qtd_matriculas_medio_integral', 'coordenadas', 'is_PEI', '_merge'],
      dtype='object')

Checagem: todos os dados são do estado de São Paulo

In [5]:
df['sigla_uf'].unique()

array(['SP'], dtype=object)

Checagem: os anos são os do período de estudo

In [6]:
df['ano_censo'].unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [7]:
df['_merge'].value_counts()

_merge
left_only    15425
both         14787
Name: count, dtype: int64

Checagem: a quantidade de escolas marcadas como PEI tem que ser igual ao merge que deu both

In [8]:
df['is_PEI'].sum() == (df['_merge']=='both').sum()

np.True_

In [9]:
matriculas_integral = [col for col in df.columns if col.endswith('_integral')]
matriculas_integral

['qtd_matriculas_infantil_integral',
 'qtd_matriculas_infantil_creche_integral',
 'qtd_matriculas_infantil_pre_escolar_integral',
 'qtd_matriculas_fundamental_integral',
 'qtd_matriculas_fundamental_anos_iniciais_integral',
 'qtd_matriculas_fundamental_anos_finais_integral',
 'qtd_matriculas_medio_integral']

In [10]:
df[matriculas_integral].max()

qtd_matriculas_infantil_integral                       0
qtd_matriculas_infantil_creche_integral                0
qtd_matriculas_infantil_pre_escolar_integral           0
qtd_matriculas_fundamental_integral                  774
qtd_matriculas_fundamental_anos_iniciais_integral    304
qtd_matriculas_fundamental_anos_finais_integral      774
qtd_matriculas_medio_integral                        598
dtype: int64

Como nennhuma escola tem infantil integral, vamos manter apenas as colunas para o médio e para o fundamental.

In [11]:
df['total_matriculas_basica_integral'] = df[matriculas_integral].sum(axis=1)

In [12]:
df.drop(columns=[col for col in matriculas_integral if 'infantil' in col], inplace=True)

In [13]:
df['possui_integral'] = df['total_matriculas_basica_integral'] > 0

In [14]:
df['possui_integral'].sum()

np.int64(7945)

In [15]:
df['total_matriculas_fundamental_integral'] = df[[col for col in matriculas_integral if 'fundamental' in col]].sum(axis=1)

In [16]:
df.columns

Index(['ano_censo', 'sigla_uf', 'nome_municipio', 'codigo_municipio',
       'codigo_escola', 'nome_escola', 'qtd_turmas_ed_basica',
       'qtd_matr_ed_basica', 'qtd_matr_medio', 'qtd_matr_fundamental',
       'qtd_matr_ed_basica_nao_declarada', 'qtd_matr_ed_basica_branca',
       'qtd_matr_ed_basica_preta', 'qtd_matr_ed_basica_parda',
       'qtd_matr_ed_basica_amarela', 'qtd_matr_ed_basica_indigena',
       'qtd_matriculas_fundamental_integral',
       'qtd_matriculas_fundamental_anos_iniciais_integral',
       'qtd_matriculas_fundamental_anos_finais_integral',
       'qtd_matriculas_medio_integral', 'coordenadas', 'is_PEI', '_merge',
       'total_matriculas_basica_integral', 'possui_integral',
       'total_matriculas_fundamental_integral'],
      dtype='object')

In [17]:
df['is_PEI'].sum()

np.int64(14787)

### Quantidade de escolas marcadas como PEI versus quantidade de escolas com matrículas no integral

Como esperado, temos mais escolas marcadas como PEI do que escolas que possuem efetivamente matrículas no integral.

Isso ocorre porque o nosso dado do PEI é de 2025. E o estudo tem como último ano 2018 (quando o PEI foi alterado). Dessa forma, temos escolas que entraram no integral posteriormente ao período de estudo e que não possuem matrículas no integral na base do censo até 2018. Então isso está como o esperado.




In [18]:
df['is_PEI'].sum() - df['possui_integral'].sum()

np.int64(6842)

### Objeto do estudo: ensino médio

No entanto, o objeto do nosso estudo é o ensino médio apenas (pois é mais fácil de comparar). Então vamos identificar agora apenas as escolas qeu tem ensino médio integral

Checando se todas as escolas ofertaram ensino medio

In [19]:
(df['qtd_matr_medio']>0).all()

np.True_

In [20]:
df['possui_integral_medio'] = df['qtd_matriculas_medio_integral']>0
df['possui_integral_medio'].sum()

np.int64(2245)

26% das escolas possuem ensino médio integral

In [21]:

df[df['possui_integral_medio']]['codigo_escola'].nunique()/df['codigo_escola'].nunique()


0.26198889449772844

In [22]:
df['is_PEI_and_possui_medio_integral'] = df['is_PEI'] & df['possui_integral_medio']
df['is_PEI_and_possui_medio_integral'].sum()

np.int64(1493)

Existem 1357 escolas que estão marcadas como PEI mas que não possuem ensino médio integral.

Novamente isso se deve ao fato das escolas terem entrado no PEI posteriormente.

In [23]:
len(set(df[df['is_PEI']]['codigo_escola'].unique()) - set(df[df['is_PEI_and_possui_medio_integral']]['codigo_escola'].unique()))

1357

Existem 458 escolas que 

In [24]:
escolas_com_integral_sem_pei = set(df[df['possui_integral_medio']]['codigo_escola'].unique()) - set(df[df['is_PEI_and_possui_medio_integral']]['codigo_escola'].unique())
print(f'QTD Escolas com integral sem PEI: {len(escolas_com_integral_sem_pei)}')

QTD Escolas com integral sem PEI: 458


In [25]:
len(escolas_com_integral_sem_pei)/df['codigo_escola'].nunique()

0.11559818273599193

In [26]:
len(escolas_com_integral_sem_pei)/df[df['is_PEI']]['codigo_escola'].nunique()

0.2364481156427465

Essas escolas representam 11% do total de escolas e 23% do total de escolas marcadas como PEI.

In [27]:
def anos_com_integral_no_medio(df):

    escolas = df['codigo_escola'].unique()
    df['primeiro_ano_integral_medio'] = None
    df['ultimo_ano_integral_medio'] = None
    df['anos_integral_medio'] = None
    for escola in escolas:
        ids_escola = df[df['codigo_escola'] == escola].index
        df_escola = df[df['codigo_escola'] == escola]
        df_escola_integral = df_escola[df_escola['possui_integral_medio']]
        anos = df_escola_integral['ano_censo'].unique()
        if anos.size>0:
            primeiro_ano = anos.min()
            ultimo_ano = anos.max()
            df.loc[ids_escola, 'primeiro_ano_integral_medio'] = primeiro_ano
            df.loc[ids_escola, 'ultimo_ano_integral_medio'] = ultimo_ano
            df.loc[ids_escola, 'anos_integral_medio'] = ','.join([str(ano) for ano in anos])

    return df


        


In [28]:
df = anos_com_integral_no_medio(df)
df['anos_integral_medio'].sample(5)

3644                    None
25414    2015,2016,2017,2018
17163                   None
12015                   2012
10585                   None
Name: anos_integral_medio, dtype: object

In [29]:
df.loc[10367]

ano_censo                                                                          2013
sigla_uf                                                                             SP
nome_municipio                                                                sao paulo
codigo_municipio                                                                3550308
codigo_escola                                                                  35003669
nome_escola                                                      alexandre von humboldt
qtd_turmas_ed_basica                                                                 12
qtd_matr_ed_basica                                                                  403
qtd_matr_medio                                                                      403
qtd_matr_fundamental                                                                  0
qtd_matr_ed_basica_nao_declarada                                                    200
qtd_matr_ed_basica_branca       

In [30]:
df['primeiro_ano_integral_medio'].max()

np.int64(2018)

In [31]:
df['ultimo_ano_integral_medio'].argmin()

np.int64(2)

In [32]:
df.loc[df['ultimo_ano_integral_medio'].argmin()]

ano_censo                                                                           2011
sigla_uf                                                                              SP
nome_municipio                                                                adamantina
codigo_municipio                                                                 3500105
codigo_escola                                                                   35031112
nome_escola                                          fleurides cavallini menechino profa
qtd_turmas_ed_basica                                                                  32
qtd_matr_ed_basica                                                                  1102
qtd_matr_medio                                                                       345
qtd_matr_fundamental                                                                 757
qtd_matr_ed_basica_nao_declarada                                                      61
qtd_matr_ed_basica_br

In [33]:
df[df['codigo_escola']==35031112][['codigo_escola', 'nome_escola', 'ano_censo', 'anos_integral_medio','qtd_matriculas_medio_integral', 'possui_integral_medio', 'is_PEI', 'is_PEI_and_possui_medio_integral']]

,codigo_escola,nome_escola,ano_censo,anos_integral_medio,qtd_matriculas_medio_integral,possui_integral_medio,is_PEI,is_PEI_and_possui_medio_integral
2,35031112,fleurides cavallini menechino profa,2011,2011,1,True,True,True
3661,35031112,fleurides cavallini menechino profa,2012,2011,0,False,True,False
7424,35031112,fleurides cavallini menechino profa,2013,2011,0,False,True,False
11190,35031112,fleurides cavallini menechino profa,2014,2011,0,False,True,False
14969,35031112,fleurides cavallini menechino profa,2015,2011,0,False,True,False
18775,35031112,fleurides cavallini menechino profa,2016,2011,0,False,True,False
22588,35031112,fleurides cavallini menechino profa,2017,2011,0,False,True,False
26411,35031112,fleurides cavallini menechino profa,2018,2011,0,False,True,False


In [34]:
def escola_com_integral_periodo_todo(df):

    escolas = df['codigo_escola'].unique()
    df['ofertou_integral_de_forma_continua'] = False
    for escola in escolas:
        ids_escola = df[df['codigo_escola'] == escola].index
        df_escola = df[df['codigo_escola'] == escola]
        if df_escola['possui_integral_medio'].max():
            primeiro_ano = int(df_escola['primeiro_ano_integral_medio'].unique()[0])
            

            ultimo_ano = int(df_escola['ultimo_ano_integral_medio'].unique()[0])
            anos = df_escola['anos_integral_medio'].unique()[0].split(',')
            anos = [int(ano) for ano in anos]

            # a escola que ofertou apenas em 2011 nao pode ser considerada como ofertando de forma continua
            if len(anos)==1 and primeiro_ano==2011:
                continue

            #ajustar o primeiro ano para nao considerar 2011 pois em 2011 nao existia o PEI
            if primeiro_ano == 2011:
                anos.remove(2011)
                primeiro_ano = min(anos)

            continuo = anos == list(range(primeiro_ano, ultimo_ano + 1))
            df.loc[ids_escola, 'ofertou_integral_de_forma_continua'] = continuo
    return df


In [35]:
df = escola_com_integral_periodo_todo(df)
df['ofertou_integral_de_forma_continua'].sum()

np.int64(6608)

In [36]:
df['ofertou_integral_de_forma_continua'].mean()

np.float64(0.21872103799814643)

In [37]:
df[df['ofertou_integral_de_forma_continua']]['codigo_escola'].nunique() / df['codigo_escola'].nunique()

0.212771327612317

In [38]:
df[df['ofertou_integral_de_forma_continua']]['codigo_escola'].nunique()/df[df['is_PEI']]['codigo_escola'].nunique()

0.4352090862157976

In [39]:
df[df['ofertou_integral_de_forma_continua']]['codigo_escola'].nunique()/df[df['possui_integral_medio']]['codigo_escola'].nunique()

0.8121387283236994

### Tratamento:

Escolas que estão marcadas como PEI e que ofertaram o integral de forma contínua nesse período até 2018

In [40]:
df['ofertou_integral_ate_2018'] = df['ultimo_ano_integral_medio'].apply(lambda x: int(x) if x else 999999) <= 2018

In [41]:
df['tratamento'] = df['ofertou_integral_de_forma_continua'] & df['is_PEI_and_possui_medio_integral'] & df['ofertou_integral_ate_2018']

Ao todo são 498 escolas com tratamento. O que represena 4% dos registros e 12% das escolas únicas.

In [42]:
df[df['tratamento']]['codigo_escola'].nunique()

498

In [43]:
df[df['tratamento']]['codigo_escola'].nunique()/ df['codigo_escola'].nunique()

0.12569409389197375

In [44]:
df['tratamento'].mean()

np.float64(0.043161657619488945)

In [45]:
df[df['tratamento']][['codigo_escola', 'nome_escola']].sample(10)

,codigo_escola,nome_escola
27007,35009854,willian rodrigues rebua professor
28167,35041497,carlos lima dias doutor
29794,35923527,luciane do espirito santo profa
21641,35001363,sao paulo
756,35924436,rodolfo jose da costa e silva
20881,35021064,jeronymo gallo monsenhor
10509,35037710,milton da silva rodrigues professor
14843,35014175,amador bueno da veiga
16860,35049712,dalva vieira itavo professora
20187,35015702,antonio berreta prof


### excluir contaminações


Escolas que fazem parte do PEI hoje e que ofertaram integral durante o período mas não ofertaram até 2018 tme um indicativo de que entraram no PEI e saíram.

O mesmo vale para escolas que porventura entraram no PEI, ofereceram integral mas em algum ano no meio não ofereceram.

Isso contamina o estudo. Temos que remover essas escolas.

df['contaminado_porque_saiu'] = df['ofertou_integral_de_forma_continua'] & df['is_PEI_and_possui_medio_integral'] & ~df['ofertou_integral_ate_2018']

In [46]:
df['contaminado_porque_saiu'] = df['ofertou_integral_de_forma_continua'] & df['is_PEI_and_possui_medio_integral'] & ~df['ofertou_integral_ate_2018']

como podemos constatar, ninguém saiu do PEI

In [47]:
df['contaminado_porque_saiu'].any()

np.False_

In [48]:
df['contaminado_porque_nao_teve_consistencia'] = df['is_PEI_and_possui_medio_integral'] & ~df['ofertou_integral_de_forma_continua']

In [49]:
df['contaminado_porque_nao_teve_consistencia'].any()

np.True_

In [50]:
df['contaminado_porque_nao_teve_consistencia'].sum()

np.int64(189)

In [51]:
df[df['contaminado_porque_nao_teve_consistencia']]['codigo_escola'].nunique()

82

In [52]:
escolas_contaminadas = df[df['contaminado_porque_nao_teve_consistencia']]['codigo_escola'].unique()

In [53]:
pd.set_option('display.max_rows', None)

In [54]:
df.columns

Index(['ano_censo', 'sigla_uf', 'nome_municipio', 'codigo_municipio',
       'codigo_escola', 'nome_escola', 'qtd_turmas_ed_basica',
       'qtd_matr_ed_basica', 'qtd_matr_medio', 'qtd_matr_fundamental',
       'qtd_matr_ed_basica_nao_declarada', 'qtd_matr_ed_basica_branca',
       'qtd_matr_ed_basica_preta', 'qtd_matr_ed_basica_parda',
       'qtd_matr_ed_basica_amarela', 'qtd_matr_ed_basica_indigena',
       'qtd_matriculas_fundamental_integral',
       'qtd_matriculas_fundamental_anos_iniciais_integral',
       'qtd_matriculas_fundamental_anos_finais_integral',
       'qtd_matriculas_medio_integral', 'coordenadas', 'is_PEI', '_merge',
       'total_matriculas_basica_integral', 'possui_integral',
       'total_matriculas_fundamental_integral', 'possui_integral_medio',
       'is_PEI_and_possui_medio_integral', 'primeiro_ano_integral_medio',
       'ultimo_ano_integral_medio', 'anos_integral_medio',
       'ofertou_integral_de_forma_continua', 'ofertou_integral_ate_2018',
       'trat

In [55]:
df_contaminadas = df[df['codigo_escola'].isin(escolas_contaminadas)].copy()
df_contaminadas.drop_duplicates(subset=['codigo_escola'], inplace=True)
df_contaminadas[['primeiro_ano_integral_medio',
       'ultimo_ano_integral_medio', 'anos_integral_medio',
       'ofertou_integral_de_forma_continua', 'ofertou_integral_ate_2018',
       'tratamento']]

,primeiro_ano_integral_medio,ultimo_ano_integral_medio,anos_integral_medio,ofertou_integral_de_forma_continua,ofertou_integral_ate_2018,tratamento
2,2011,2011,2011,False,True,False
23,2013,2015,"2013,2015",False,True,False
181,2011,2011,2011,False,True,False
196,2011,2011,2011,False,True,False
198,2011,2011,2011,False,True,False
199,2011,2011,2011,False,True,False
243,2012,2018,"2012,2014,2015,2016,2017,2018",False,True,False
266,2011,2011,2011,False,True,False
274,2011,2011,2011,False,True,False
322,2011,2011,2011,False,True,False


Removendo as contaminadas

In [56]:
df = df[~df['codigo_escola'].isin(escolas_contaminadas)].copy()

In [57]:
df.columns

Index(['ano_censo', 'sigla_uf', 'nome_municipio', 'codigo_municipio',
       'codigo_escola', 'nome_escola', 'qtd_turmas_ed_basica',
       'qtd_matr_ed_basica', 'qtd_matr_medio', 'qtd_matr_fundamental',
       'qtd_matr_ed_basica_nao_declarada', 'qtd_matr_ed_basica_branca',
       'qtd_matr_ed_basica_preta', 'qtd_matr_ed_basica_parda',
       'qtd_matr_ed_basica_amarela', 'qtd_matr_ed_basica_indigena',
       'qtd_matriculas_fundamental_integral',
       'qtd_matriculas_fundamental_anos_iniciais_integral',
       'qtd_matriculas_fundamental_anos_finais_integral',
       'qtd_matriculas_medio_integral', 'coordenadas', 'is_PEI', '_merge',
       'total_matriculas_basica_integral', 'possui_integral',
       'total_matriculas_fundamental_integral', 'possui_integral_medio',
       'is_PEI_and_possui_medio_integral', 'primeiro_ano_integral_medio',
       'ultimo_ano_integral_medio', 'anos_integral_medio',
       'ofertou_integral_de_forma_continua', 'ofertou_integral_ate_2018',
       'trat

In [58]:
df['contaminado_porque_saiu'].any()

np.False_

In [59]:
df['contaminado_porque_nao_teve_consistencia'].any()

np.False_

In [60]:
remover_colunas = ['contaminado_porque_nao_teve_consistencia', 'contaminado_porque_saiu']
df.drop(columns=remover_colunas, inplace=True)
df.columns

Index(['ano_censo', 'sigla_uf', 'nome_municipio', 'codigo_municipio',
       'codigo_escola', 'nome_escola', 'qtd_turmas_ed_basica',
       'qtd_matr_ed_basica', 'qtd_matr_medio', 'qtd_matr_fundamental',
       'qtd_matr_ed_basica_nao_declarada', 'qtd_matr_ed_basica_branca',
       'qtd_matr_ed_basica_preta', 'qtd_matr_ed_basica_parda',
       'qtd_matr_ed_basica_amarela', 'qtd_matr_ed_basica_indigena',
       'qtd_matriculas_fundamental_integral',
       'qtd_matriculas_fundamental_anos_iniciais_integral',
       'qtd_matriculas_fundamental_anos_finais_integral',
       'qtd_matriculas_medio_integral', 'coordenadas', 'is_PEI', '_merge',
       'total_matriculas_basica_integral', 'possui_integral',
       'total_matriculas_fundamental_integral', 'possui_integral_medio',
       'is_PEI_and_possui_medio_integral', 'primeiro_ano_integral_medio',
       'ultimo_ano_integral_medio', 'anos_integral_medio',
       'ofertou_integral_de_forma_continua', 'ofertou_integral_ate_2018',
       'trat

In [61]:
df[df['tratamento']]['codigo_escola'].nunique() / df['codigo_escola'].nunique()

0.1283505154639175

In [62]:
df[df['tratamento']]['codigo_escola'].nunique() / df[df['possui_integral_medio']]['codigo_escola'].nunique()

0.5209205020920502

In [63]:
df[df['tratamento']]['codigo_escola'].nunique() / df[df['ofertou_integral_de_forma_continua']]['codigo_escola'].nunique()

0.5907473309608541

In [65]:
from utils.save_csv import save_csv
save_csv(df, 'dados_limpos.csv')

Data saved to data/dados_limpos.csv
